A permutation $\pi$ of $X$ is a bijective function from $X \to X$. The set of all permutations of the set $X = \{1, \dots, n\}$ is the symmetric group $S_n$.
If $\pi$ is a permutation and $y = \pi y$, then $y$ is called a fixed point of $\pi$.

In [28]:
class Permutation:
    '''
    A class to represent a permutation of the set {1, 2, ..., n}.
    This class uses 1-based indexing for accessing elements, to align with
    standard mathematical notation. For a permutation object p, p[i] will
    return the value that i is mapped to.

    Attributes:
        _p: A 0-indexed list storing the permutation. _p[i-1] = π(i).
        size: The number of elements, n, in the permutation.
    '''

    def __init__(self, p_list):
        '''
        Initializes the Permutation object.
        Args:
            p_list: A list of integers representing the permutation.
        '''
        # Internal storage is a 0-indexed list
        self._p = list(p_list)
        self.size = len(p_list)

    def __len__(self):
        '''Returns the size (n) of the permutation.'''
        return self.size

    def __repr__(self):
        '''Provides a string representation for printing the permutation.'''
        return f"Perm({self._p})"

    def __eq__(self, other):
        '''Checks if two Permutation objects are equal.'''
        if not isinstance(other, Permutation):
            return NotImplemented
        return self._p == other._p

    def __getitem__(self, i):
        '''
        Gets the value that i is mapped to, using 1-based indexing.
        Args:
            i: The element to map (from 1 to n).
        Returns:
            The value π(i).
        '''
        if not 1 <= i <= self.size:
            raise IndexError(f"Index {i} is out of bounds for a permutation of size {self.size}.")
        # Convert 1-based index to 0-based for internal list access
        return self._p[i - 1]

    def __mul__(self, other):
        '''
        Computes the product of two permutations (self * other).
        Args:
            other: The second permutation in the product.

        Returns:
            A new Permutation object representing the product.
        '''
        if not isinstance(other, Permutation) or self.size != other.size:
            raise ValueError("Permutations must be of the same size to compute their product.")

        new_p_list = [0] * self.size
        for i in range(1, self.size + 1):
            # The product maps i -> self(other(i)).
            # We use our 1-based __getitem__ to make this line intuitive.
            new_p_list[i - 1] = self[other[i]]

        return Permutation(new_p_list)

    def inverse(self):
        '''
        Computes the inverse of this permutation.
        Returns:
            Permutation: A new Permutation object representing the inverse.
        '''
        inv_p_list = [0] * self.size
        for i in range(1, self.size + 1):
            # If self maps i to j, then the inverse must map j to i.
            # self[i] gives us the value j.
            j = self[i]
            # In the inverse list, we set the value at index (j-1) to be i.
            inv_p_list[j - 1] = i

        return Permutation(inv_p_list)

pi1 = Permutation([2, 4, 1, 3])
pi2 = Permutation([3, 2, 4, 1])

print(f"π₁ = {pi1}")
print(f"π₂ = {pi2}\n")

print("--- Demonstration of 1-based Indexing ---")
print(f"π₁(3) is accessed as pi1[3], which gives: {pi1[3]}")
print(f"π₂(1) is accessed as pi2[1], which gives: {pi2[1]}\n")

print("--- Procedure: Compute Product ---")
print(f"The product π₁ * π₂ is: {pi1 * pi2}\n")

print("--- Procedure: Compute Inverse ---")
print(f"The inverse π₁⁻¹ is: {pi1.inverse()}\n")

identity = pi1 * pi1.inverse()
print(f"Verification: π₁ * π₁⁻¹ = {identity}")

π₁ = Perm([2, 4, 1, 3])
π₂ = Perm([3, 2, 4, 1])

--- Demonstration of 1-based Indexing ---
π₁(3) is accessed as pi1[3], which gives: 1
π₂(1) is accessed as pi2[1], which gives: 3

--- Procedure: Compute Product ---
The product π₁ * π₂ is: Perm([1, 4, 3, 2])

--- Procedure: Compute Inverse ---
The inverse π₁⁻¹ is: Perm([3, 1, 4, 2])

Verification: π₁ * π₁⁻¹ = Perm([1, 2, 3, 4])


The complexity of the computing the inverse of a permutation is $O(n)$, where $n$ is the length of the permutation since we iterate once through each element and each step inside the loop is a constant-time operation to construct the inverse element.

Suppose the permutation group $G$ is generated by permutations $\pi_1, \dots, \pi_k$. First we reduce the number of generators with the stripping algorithm of Sims.

Let $A$ be an $n \times n$ array of permutations which is initially empty. Suppose we have already put the first $l - 1$ permutations into the array. If $\pi_l$ does not fix $1$ and the $\pi_l(1)$-th entry in the first row is still empty then put $\pi_l$ there. Suppose the $\pi_l(1)$-th entry is the permutation $g$. Then modify $\pi_l$ to be $g^{-1}\pi_l$ so the new $\pi_l$ fixes $1$. Go to the second row.

If $\pi_l$ does not fix $2$ and the $\pi_l(2)$-th entry in the second row is still empty then put $\pi_l$ there. If the entry is $g$ then modify $\pi_l$ to be $g^{-1}\pi_l$ which fixes $1$ and $2$. Continue this process.

If we reach the last row then we must have produced the trivial permutation which can be omitted from the generating set. Once a permutation is placed in the array, or deemed to be the trivial permutation, we go on to try to place the next permutation in the array.

---

The algorithm refines a given set of generators, $S = \{\pi_1, \dots, _k\}$, for a group $G$. It uses an $n \times n$ array $A$, which acts as a lookup table for a new, smaller set of generators $S'$. A generator $g$ that fixes elements $1, \dots, i-1$ and maps $i$ to $j$ can be stored in $A[i-1][j-1]$.

The algorithm for each generator $\pi_l$ in S is as follows:
1.  Let $p = \pi_l$.
2.  For $i$ from $1$ to $n-1$:
    *   If $p$ does not fix $i$ (i.e., $p(i) \neq i$), then let $j = p(i)$.
    *   Check the array entry $A[i-1][j-1]$:
        *   If $A[i-1][j-1]$ is empty, then place $p$ in $A[i-1][j-1]$. This permutation $p$ is now part of our new generating set $S'$. We are done with $\pi_l$ and move to the next original generator $\pi_{l+1}$.
    *   If $A[i-1][j-1]$ contains a generator $g$, then update $p$ by setting it to $g^{-1}p$. This new $p$ now fixes $i$ (because $g^{-1}(p(i)) = g^{-1}(j) = i$), in addition to fixing $1, \dots, i-1$. We continue the loop to check $i+1$ with this updated $p$.
3.  If the loop finishes (meaning $p$ has been modified to fix $1, \dots, n-1$), then $p$ has become the identity permutation. This original generator $\pi_l$ does not add a new generator to $S'$. We move to $\pi_{l+1}$.

Proof that $\langle S' \rangle \subseteq \langle S \rangle$. Any generator $s' \in S'$ is of the form $s' = (g_m^{-1}) \cdots (g_1^{-1}) \pi_l$, where $g_1, \dots, g_m$ are generators already in $S'$ By induction, we can argue that every $g_i \in \langle S \rangle$. The base cases are the first generators added, which are unmodified versions of elements from $S$. Since $\pi_l \in \langle S \rangle$ and each $g_i \in \langle S \rangle$, any product of these elements and their inverses must also be in $\langle S \rangle$. Therefore, every generator $s' \in S'$ is an element of $\langle S \rangle$. This implies that $\langle S' \rangle \leq \langle S \rangle$.

Proof that $\langle S \rangle \subseteq \langle S' \rangle$. Consider the processing of an arbitrary $\pi_l$. There are two cases. Either the process adds a new generator $p$ to $S'$, so $p = (g_m^{-1}) \cdots (g_1^{-1}) \pi_lₗ$, where each $g_i \in S'$. We can rearrange this to show $\pi_l$ is a product of elements in $\langle S' \rangle$, hence is itself in $\langle S' \rangle$. Alternatively, the process reduces $\pi_l$ to $Id = (g_m^{-1}) \cdots (g_1^{-1}) \pi_l$. This again shows that $\pi_l \in \langle S' \rangle$. Since all generators of $G$ are in $\langle S' \rangle$, then $\langle S \rangle \leq \langle S' \rangle$.



In [29]:
import copy

def sims_stripping_algorithm(initial_generators):
    '''
    Computes a reduced set of generators for a permutation group using
    Sims' Stripping Algorithm.
    Args:
        initial_generators: The initial set of generators.
    Returns:
        A new set of generators in the reduced form.
    '''
    if not initial_generators:
        return []

    n = initial_generators[0].size
    # Initialize an n x n array (using 0-indexing for rows 0 to n-1)
    gen_array = [[None for _ in range(n)] for _ in range(n)]

    for gen in initial_generators:
        p = copy.deepcopy(gen) # Work with a copy of the generator
        for i in range(1, n + 1):
            j = p[i]
            if j == i:
                continue
            g = gen_array[i - 1][j - 1]
            if g is None:
                gen_array[i - 1][j - 1] = p
                break
            else:
                p = g.inverse() * p

    reduced_generators = []
    for i in range(n):
        for j in range(n):
            if gen_array[i][j] is not None:
                reduced_generators.append(gen_array[i][j])
    return reduced_generators

print("--- Example 1: The Symmetric Group S3 ---")
print("S3 can be generated by a transposition (1 2) and a 3-cycle (1 2 3).")
pi1 = Permutation([2, 1, 3])
pi2 = Permutation([2, 3, 1])
initial_set_s3 = [pi1, pi2]
print(f"Initial generators: {initial_set_s3}")
reduced_set_s3 = sims_stripping_algorithm(initial_set_s3)
print(f"Resulting reduced set of generators: {reduced_set_s3}")

print("\n--- Example 2: The Dihedral Group D8 ---")
print("D8 can be generated by a rotation (1 2 3 4) and a reflection (1 3).")
rotation = Permutation([2, 3, 4, 1])
reflection = Permutation([3, 2, 1, 4])
initial_set_d8 = [rotation, reflection]
print(f"Initial generators: {initial_set_d8}")
reduced_set_d8 = sims_stripping_algorithm(initial_set_d8)
print(f"Resulting reduced set of generators: {reduced_set_d8}")

print("\n--- Example 3: A Redundant Generating Set ---")
print("Add a redundant generator to the S3 set: (1 3) = (1 2)(1 2 3).")
pi3 = Permutation([3, 2, 1])
redundant_set_s3 = [pi1, pi2, pi3]
print(f"Initial generators: {redundant_set_s3}")
reduced_redundant_s3 = sims_stripping_algorithm(redundant_set_s3)
print(f"Resulting reduced set of generators: {reduced_redundant_s3}")

--- Example 1: The Symmetric Group S3 ---
S3 can be generated by a transposition (1 2) and a 3-cycle (1 2 3).
Initial generators: [Perm([2, 1, 3]), Perm([2, 3, 1])]
Resulting reduced set of generators: [Perm([2, 1, 3]), Perm([1, 3, 2])]

--- Example 2: The Dihedral Group D8 ---
D8 can be generated by a rotation (1 2 3 4) and a reflection (1 3).
Initial generators: [Perm([2, 3, 4, 1]), Perm([3, 2, 1, 4])]
Resulting reduced set of generators: [Perm([2, 3, 4, 1]), Perm([3, 2, 1, 4])]

--- Example 3: A Redundant Generating Set ---
Add a redundant generator to the S3 set: (1 3) = (1 2)(1 2 3).
Initial generators: [Perm([2, 1, 3]), Perm([2, 3, 1]), Perm([3, 2, 1])]
Resulting reduced set of generators: [Perm([2, 1, 3]), Perm([3, 2, 1]), Perm([1, 3, 2])]


The modified set of generators $S'$ consists of the permutations placed into the $n \times n$ array $A$. A generator is only placed in an entry if that entry is empty. Therefore, the maximum size of $S'$ is the total number of available slots in $A$.

Consider a generator $g$ stored at $A[i-1][j-1]$. Then $g$ fixes the elements $\{1, 2, \dots, i-1\}$ and $g(i) = j > i$. The stripping loop for $i$ runs from $1$ to $n-1$ and the number of available slots for each row $i$ is $n-i$, so the total number of slots is the sum of the first $n-1$ positive integers, which is given by $(n-1)((n-1)+1) / 2 = n(n-1)/2 = O(n^2)$.

Let $k$ be the size of the original generating set and $n$ be the degree of the permutation group. Basic permutation operations such as taking the product, inverse and storage are $O(n)$.

The algorithm consists of an outer loop that runs $k$ times (once for each original generator). For each of these $k$ generators, we perform the stripping process. The stripping process involves an inner loop that iterates from $i = 1$ to at worst $n-1$, where all generators are fully stripped to the identity and are not placed into the array.

The inner loop runs $n-1$ times. In each iteration $i$ of this loop where a modification occurs, we perform an inverse and a product so the total cost for one step of modification is $O(n)$. The maximum cost to process a single original generator is multiplies this by the number of steps, so costs $O(n^2)$. Since there are $k$ original generators, the total number of operations is $O(kn^2)$.